# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
data_weather = pd.read_csv("../WeatherPy/Weather.csv")
data_weather

,Cities,Country,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,shelburne,CA,44.08,-80.20,90,87,57.99,8.19
1,nikolskoye,RU,59.70,30.79,75,76,51.80,6.71
2,lishui,CN,28.46,119.91,45,51,93.36,3.00
3,ribeira grande,PT,38.52,-28.70,43,62,66.58,12.41
4,taolanaro,,0.00,0.00,0,0,0.00,0.00
...,...,...,...,...,...,...,...,...
545,ossora,RU,59.24,163.07,28,67,50.16,4.14
546,punta de bombon,PE,-17.17,-71.79,96,84,60.19,7.54
547,pingyin,CN,36.28,116.45,66,28,79.18,7.99
548,visnes,NO,59.36,5.22,77,76,57.00,13.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:

#Access maps with API key
gmaps.configure(api_key = g_key)

#Lat and Lon in Locations and Humidity in humidity
locations = data_weather[["Latitude", "Longitude"]]
humidity = data_weather["Humidity"].astype(float)
humidity_max = data_weather["Humidity"].max()


In [41]:
#store gmaps as a variable
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
data_weather

,Cities,Country,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,shelburne,CA,44.08,-80.20,90,87,57.99,8.19
1,nikolskoye,RU,59.70,30.79,75,76,51.80,6.71
2,lishui,CN,28.46,119.91,45,51,93.36,3.00
3,ribeira grande,PT,38.52,-28.70,43,62,66.58,12.41
4,taolanaro,,0.00,0.00,0,0,0.00,0.00
...,...,...,...,...,...,...,...,...
545,ossora,RU,59.24,163.07,28,67,50.16,4.14
546,punta de bombon,PE,-17.17,-71.79,96,84,60.19,7.54
547,pingyin,CN,36.28,116.45,66,28,79.18,7.99
548,visnes,NO,59.36,5.22,77,76,57.00,13.87


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
df_hotel = data_weather.loc[(data_weather["Wind Speed"] <= 10) 
                         & (data_weather["Cloudiness"] == 0) 
                         & (data_weather["Max Temperature"] >= 70)
                         & (data_weather["Max Temperature"] <= 80)]
df_hotel.shape
df_hotel.reset_index(inplace=True,drop=True)
df_hotel

,Cities,Country,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,hervey bay,AU,-25.30,152.85,0,60,75.99,7.00
1,jadu,LY,31.95,12.03,0,43,70.00,7.38
2,wuwei,CN,37.93,102.64,0,18,73.72,7.23
3,dunda,TZ,-8.12,34.25,0,41,75.18,4.18
4,port macquarie,AU,-31.43,152.92,0,52,79.00,8.59
5,sao felix do xingu,BR,-6.64,-51.99,0,44,71.20,0.74
6,maneadero,MX,31.72,-116.57,0,76,73.00,1.01
7,ridgecrest,US,35.62,-117.67,0,18,77.00,4.23
8,uruzgan,AF,32.83,66.00,0,17,74.16,2.53
9,milan,IT,45.46,9.19,0,88,71.01,1.12


In [52]:
params={"radius":5000,"type":"hotel","keyword":"hotels","key":g_key}
for index,row in df_hotel.iterrows():
    print("Searching for {}".format(row['Cities']))
    lat=row['Latitude']
    lon=row['Longitude']
    params['location'] = f'{lat},{lon}'
    print(params['location'])
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    
    response=requests.get(base_url,params=params).json()
    
    try:
        df_hotel.loc[index,'Hotel_Name']=response['results'][0]['name']
    except KeyError:
        print("Couldn't find results")
    except IndexError:
        print("No hotels were found in the vicinity")

Searching for hervey bay
-25.3,152.85


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Searching for jadu
31.95,12.03
No hotels were found in the vicinity
Searching for wuwei
37.93,102.64
Searching for dunda
-8.12,34.25
No hotels were found in the vicinity
Searching for port macquarie
-31.43,152.92
Searching for sao felix do xingu
-6.64,-51.99
Searching for maneadero
31.72,-116.57
No hotels were found in the vicinity
Searching for ridgecrest
35.62,-117.67
Searching for uruzgan
32.83,66.0
No hotels were found in the vicinity
Searching for milan
45.46,9.19
Searching for antibes
43.56,7.13
Searching for manbij
36.53,37.95
Searching for sao miguel do araguaia
-13.28,-50.16


In [53]:
df_hotel

,Cities,Country,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed,Hotel_Name
0,hervey bay,AU,-25.30,152.85,0,60,75.99,7.00,Oaks Hervey Bay Resort and Spa
1,jadu,LY,31.95,12.03,0,43,70.00,7.38,NaN
2,wuwei,CN,37.93,102.64,0,18,73.72,7.23,Xiliang Hotel
3,dunda,TZ,-8.12,34.25,0,41,75.18,4.18,NaN
4,port macquarie,AU,-31.43,152.92,0,52,79.00,8.59,ibis Styles Port Macquarie
5,sao felix do xingu,BR,-6.64,-51.99,0,44,71.20,0.74,Apart Hotel Rio Xingu
6,maneadero,MX,31.72,-116.57,0,76,73.00,1.01,NaN
7,ridgecrest,US,35.62,-117.67,0,18,77.00,4.23,SpringHill Suites by Marriott Ridgecrest
8,uruzgan,AF,32.83,66.00,0,17,74.16,2.53,NaN
9,milan,IT,45.46,9.19,0,88,71.01,1.12,VIU Hotel Milan


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>Cities</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotel.iterrows()]
locations = df_hotel[["Latitude", "Longitude"]]

In [63]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))